In [102]:
# explore stimulation data
import os
from cmlreaders import CMLReader, get_data_index
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [109]:
# get subject and experiment data
experiment = 'FR3'
rootdir = '/'
df = get_data_index("r1", rootdir = rootdir)
df = df[(df['experiment'] == experiment ) | (df['experiment'] ==  'cat' + experiment )]
df[['subject', 'session', 'experiment']]
len(df['subject'].unique())

26

In [110]:
# get item-level data 
report_db_location = "/scratch/report_database/"
import os
import glob

from ramutils.reports.summary import FRStimSessionSummary
fr_session_summary_locations = glob.glob(os.path.join(report_db_location, '*' + experiment + '*session_summary*'))
session_summaries = []

for summary_loc in fr_session_summary_locations:
    summary = FRStimSessionSummary.from_hdf(summary_loc)
    session_summaries.append(summary)

combined_item_df = FRStimSessionSummary.combine_sessions(session_summaries)
combined_item_df.head()

# combined_item_df[covars]
combined_item_df['is_tbi'] = False
combined_item_df['is_LTC'] = False
combined_item_df['amplitude'] = combined_item_df['amplitude'].astype('float')

/home2/tungphan/.conda/envs/supereeg/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
/home2/tungphan/.conda/envs/supereeg/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
/home2/tungphan/.conda/envs/supereeg/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
/home2/tungphan/.conda/envs/supereeg/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
/home2/tungphan/.conda/envs/supereeg/lib/python3.6/site-packages/h5py/_hl/dataset.py:313

In [113]:
len(combined_item_df['subject'].unique()) == len(df['subject'].unique()) # check if the number of subjects is the same 

True

In [114]:
# get all unique subjects
subjects = combined_item_df['subject'].unique()

In [115]:
for subject in subjects:
    try:
        df_subject = combined_item_df[combined_item_df['subject'] == subject]  # select data for a particular subject
        exp_sessions = df_subject.groupby(['experiment','session']).size().reset_index() # grab experiment session combo

        for exp_sess in exp_sessions.values:  # loop 
            exp, session = exp_sess[0], exp_sess[1]
            df_subject_session = df_subject[(df_subject['session'] == session) & (df_subject['experiment'] == exp)]
            if np.nanmax(df_subject_session.amplitude) < 1.0:
                stim_items = df_subject_session[df_subject_session['is_stim_item'] == 1]
                locations = stim_items['location'].unique()
                locations = locations[locations != 'nan']
#                 print(locations)
                locations = locations[0].lower()
                is_LTC = ('left' in locations) and ('temporal' in locations) # select subjects with LTC stimulation
                df_subject.loc[df_subject.session == session, 'is_LTC'] = is_LTC
                combined_item_df.loc[combined_item_df.subject == subject, 'is_LTC'] = df_subject.is_LTC
    except:
        print("{}'s data does not exist".format(subject))  # print subject with error

In [116]:
tbi_frame = pd.read_csv('TBI_info.csv')
tbi_definite_subjects = tbi_frame['Subject'].unique()
tbi_possibly_subjects = ['R1008J', 'R1014D', 'R1016M', 'R1019J', 'R1047D', 'R1067E', 'R1156D', 'R1162N', 'R1195E', 'R1201P', 'R1212P', 'R1337E', 'R1339D', 'R1345D', 'R1379E']
excluded_subjects = ['R1124J','R1145J','R1154D','R1161E','R1163T','R1166D','R1170J','R1195E','R1200T','R1201P','R1202M','R1204T','R1217T','R1223E','R1226D','R1230J','R1235E','R1236J','R1243T','R1247P','R1260D','R1264P','R1274T','R1277J','R1286J']
tbi_subjects = np.append(tbi_definite_subjects, tbi_possibly_subjects)
tbi_mask = np.array([x in tbi_subjects for x in combined_item_df['subject']])

excluded_mask = np.array([x not in excluded_subjects for x in combined_item_df['subject']])
tbi_mask = tbi_mask & excluded_mask

combined_item_df = combined_item_df[tbi_mask & combined_item_df.is_LTC]
combined_item_df['is_tbi'] = True
combined_item_df = combined_item_df[combined_item_df['list'] > 3]  # exclude first 3 lists
today = pd.Timestamp.today().date()  # get date
combined_item_df.to_csv(experiment + '_all_amp_' +  str(today) + '.csv')

In [117]:
tbi_subjects

array(['R1004D', 'R1021D', 'R1027J', 'R1041M', 'R1063C', 'R1064E',
       'R1087N', 'R1102P', 'R1105E', 'R1130M', 'R1144E', 'R1164E',
       'R1169P', 'R1221P', 'R1227T', 'R1233E', 'R1235E', 'R1276D',
       'R1294C', 'R1317D', 'R1374T', 'R1465D', 'R1487T', 'R1488T',
       'R1489E', 'R1491T', 'R1008J', 'R1014D', 'R1016M', 'R1019J',
       'R1047D', 'R1067E', 'R1156D', 'R1162N', 'R1195E', 'R1201P',
       'R1212P', 'R1337E', 'R1339D', 'R1345D', 'R1379E'], dtype=object)